In [1]:
from src.all_in_one import *

In [2]:
ref = pd.read_excel(r'D:\fc-performeter\notebooks\referrals.xlsx', engine='openpyxl')

C:\Users\myagh\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
# Mapping referral directions
ref.insert(6, 'ext_ref', ref['OutgoingInternalIngoing'].apply(lambda x: 'Yes' if x == 'outgoing' else 'No' )) # cases referred from ADMSP to Others
ref.insert(7, 'int_ref', ref['OutgoingInternalIngoing'].apply(lambda x: 'Yes' if x == 'internal' else 'No' )) # cases internally referred
ref.insert(8, 'rec_ref', ref['OutgoingInternalIngoing'].apply(lambda x: 'Yes' if x == 'ingoing' else 'No' ))  # cases received from others

In [4]:
# Choose what columns are not needed here
ref = ref.drop(columns=['age', 'country', 'personal/familyMember', 'familyMemberNumber', 'group/individual', 'OutgoingInternalIngoing', 'note'])

In [5]:
# columns renaming
ref.rename(columns={
    'referral_ID': 'rid',
    'FC_ID': 'fcid',
    'beneficiaryName': 'ben_name',
    'gender': 'sex',
    'socialAssessmentDate': 'social_ass_date',
    'referralDate': 'ref_date',
    'referredToFromEntity': 'entity',
}, inplace=True)

In [6]:
# Type transformation
ref['fcid'] = ref['fcid'].astype("Int64")
ref['ref_date'] = pd.to_datetime(ref['ref_date'])

In [7]:
ref.head()

,#,rid,fcid,ben_name,sex,ext_ref,int_ref,rec_ref,need,social_ass_date,ref_date,entity
0,1,R0007,8110020016,نور أشقر,Female,No,No,No,Other,NaT,2020-01-13,UOSSM
1,2,R0007,8110020016,نور أشقر,Female,No,No,No,Other,NaT,2020-02-19,IDA
2,3,NaN,8110030014,سهام حمام,Female,No,No,No,MHPSS,NaT,2020-01-31,SAMS
3,4,R0001,8110020026,كوثر العيسى,Female,No,No,No,Other,NaT,2020-02-04,IDA
4,5,R0019,8110020031,دينا حياني,Female,No,No,No,MHPSS,NaT,2020-02-19,SAMS


In [8]:
ref['entity'].value_counts()

entity
TMH                          143
Medenıyet köprüsü derneğı    117
WHH                          110
ADMSP                        104
Orange                       102
SAMS                          96
TKV                           31
CARE                          30
women support                 28
Qatar Charity                 28
IDA                           21
SJAC                          10
Mercy without limits          10
TAAFI                          8
HUMAN APPEAL                   7
UOSSM                          6
Önder                          5
ATAA                           5
LD                             4
EMHRF                          3
Syrian Community               3
FC_LD                          3
Enaraa                         3
ÖNDER                          2
UNHCR                          2
Onsor                          2
Care                           2
bonyan                         1
bülbülzada                     1
Legal Clinic                   1
Aar

In [9]:
# separating dataframe upon referral types
ext_ref = ref[ref['ext_ref'] == 'Yes'] 

In [10]:
ext_ref = ext_ref.drop(columns=['int_ref', 'rec_ref'])

In [13]:
ext_ref['ref_date'] = pd.to_datetime(ext_ref['ref_date'], errors='coerce')

In [16]:
# Define the date range
start_date = pd.to_datetime('2024-01-01')
end_date = pd.to_datetime('2024-06-30')

filtered_ext_ref = filter_dataframe_on_date(ext_ref, ['ref_date'], start_date, end_date, filter_type=FilterType.IN)
# Create a mask to filter rows on dates required
# mask = ext_ref['ref_date'].apply(lambda row: any(start_date <= date <= end_date for date in row if pd.notnull(date)), axis=1)

# Filter the DataFrame using the mask
# filtered_ext_ref = ext_ref[mask]

In [ ]:
# ext_ref[ext_ref.duplicated(subset=['rid'], keep=False)]

In [ ]:
# int_ref = ref[ref['int_ref'] == 'Yes']
# int_ref = int_ref.drop(columns=['ext_ref', 'rec_ref'])

In [ ]:
# rec_ref = ref[ref['rec_ref'] == 'Yes']
# rec_ref = rec_ref.drop(columns=['ext_ref', 'int_ref'])

In [17]:
filtered_ext_ref

,#,rid,fcid,ben_name,sex,ext_ref,need,social_ass_date,ref_date,entity
684,685,R1174,<NA>,غياث الأمين,Male,Yes,MHPSS,NaT,2024-01-05,SAMS
685,686,R0406,8110050045,ابتسام غزال,Female,Yes,Physical Therapy,2023-07-12,2024-01-05,SAMS
686,687,R1125,<NA>,سهام الصيرفي,Female,Yes,MHPSS,NaT,2024-01-08,TMH
687,688,R1161,<NA>,عماد بكداش,Male,Yes,MHPSS,NaT,2024-01-08,TMH
688,689,R1162,<NA>,محمد المحلي,Male,Yes,MHPSS,NaT,2024-01-08,TMH
...,...,...,...,...,...,...,...,...,...,...
853,854,R0353,8110020099,دلال الشيخ,Female,Yes,Cash assistance,NaT,2024-05-24,Qatar Charity
854,855,R0518,8110090004,عائشة المصطفى,Female,Yes,Cash assistance,NaT,2024-05-24,Qatar Charity
855,856,R0298,8110020089,يسرى الحويل,Female,Yes,Cash assistance,NaT,2024-05-24,Qatar Charity
856,857,R0460,8110090003,سها الحسن,Female,Yes,Cash assistance,NaT,2024-05-24,Qatar Charity


In [22]:
total_number_of_ref = len(filtered_ext_ref)
total_number_of_ref

168

In [25]:
unique_ref_beneficiaries = filtered_ext_ref.drop_duplicates(subset=['rid'])
total_number_ref_beneficiaries = len(unique_ref_beneficiaries)

In [26]:
total_number_ref_beneficiaries

142

In [27]:
unique_ref_beneficiaries['sex'].value_counts()

sex
Female    103
Male       39
Name: count, dtype: int64

In [29]:
filtered_ext_ref['need'].value_counts()


need
Cash assistance        118
MHPSS                   43
Legal documentation      5
Physical Therapy         1
Legal assistance         1
Name: count, dtype: int64

In [30]:
filtered_ext_ref['entity'].value_counts()

entity
Orange           60
TMH              33
WHH              30
Qatar Charity    28
SAMS             11
SJAC              5
whh               1
Name: count, dtype: int64

In [33]:
unique_ref_beneficiaries.to_csv('unique_ref_beneficiaries.csv', index=False)